# 1. Instalar e Importar Librerías Necesarias

En esta sección instalaremos e importaremos todas las librerías necesarias para el procesamiento de PDFs, vectorización, tokenización y uso de modelos ligeros multilenguaje compatibles con Ollama.

In [ ]:
# Instalar librerías necesarias
!pip install PyPDF2 sentence-transformers langchain faiss-cpu ollama

# Importar librerías
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle

# 2. Cargar y Leer Múltiples PDFs

Esta sección permite seleccionar y leer múltiples archivos PDF desde una carpeta, extrayendo el texto de cada uno sin límite de cantidad.

In [ ]:
# Definir la carpeta donde están los PDFs
pdf_folder = "c:/Users/TZ002NB/Desktop/rag-fine-tunning/pdfs"

# Obtener la lista de archivos PDF
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]

# Extraer texto de todos los PDFs
def extract_text_from_pdfs(pdf_files):
    all_texts = []
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
        all_texts.append(text)
    return all_texts

pdf_texts = extract_text_from_pdfs(pdf_files)
print(f"Se extrajo texto de {len(pdf_texts)} archivos PDF.")

# 3. Tokenizar y Vectorizar el Texto Extraído

Utilizaremos un modelo multilenguaje ligero de `sentence-transformers` para tokenizar y vectorizar el texto extraído de los PDFs.

In [ ]:
# Cargar modelo multilenguaje ligero
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"
embedder = SentenceTransformer(model_name)

# Vectorizar los textos (puedes dividir en chunks si los textos son muy largos)
def chunk_text(text, max_length=512):
    # Divide el texto en fragmentos de longitud máxima
    words = text.split()
    return [' '.join(words[i:i+max_length]) for i in range(0, len(words), max_length)]

all_chunks = []
for doc in pdf_texts:
    all_chunks.extend(chunk_text(doc))

# Obtener los vectores
embeddings = embedder.encode(all_chunks, show_progress_bar=True, convert_to_numpy=True)
print(f"Se generaron {embeddings.shape[0]} vectores.")

# 4. Cargar un Modelo Multilenguaje Ligero Compatible con Ollama

En esta sección se muestra cómo cargar un modelo multilenguaje ligero que pueda ejecutarse localmente y sea compatible con Ollama.

In [ ]:
# Ejemplo de carga de modelo con Ollama (requiere tener Ollama instalado y corriendo)
# Puedes usar modelos ligeros como 'llama2', 'mistral', 'phi', etc.

import subprocess

def ollama_pull(model_name="phi"):
    result = subprocess.run(["ollama", "pull", model_name], capture_output=True, text=True)
    print(result.stdout)

# Descargar modelo ligero (ejemplo: phi)
ollama_pull("phi")

# 5. Agregar los Vectores al Modelo para Indexación

Utilizaremos FAISS para indexar los vectores y permitir búsquedas semánticas y consultas posteriores.

In [ ]:
# Crear el índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Guardar los chunks para referencia posterior
with open("c:/Users/TZ002NB/Desktop/rag-fine-tunning/chunks.pkl", "wb") as f:
    pickle.dump(all_chunks, f)

# 6. Guardar el Modelo Vectorizado para Uso Posterior

Guardaremos el índice FAISS y los textos fragmentados para poder consultarlos desde otros archivos o proyectos.

In [ ]:
# Guardar el índice FAISS
faiss.write_index(index, "c:/Users/TZ002NB/Desktop/rag-fine-tunning/faiss_index.bin")

# Guardar los embeddings por si se requieren
np.save("c:/Users/TZ002NB/Desktop/rag-fine-tunning/embeddings.npy", embeddings)

# 7. Ejemplo de Compilación del Proyecto

Puedes empaquetar este proyecto como un módulo o script para reutilizarlo en otros entornos. Por ejemplo, puedes crear un archivo `procesar_pdfs.py` con las funciones de extracción, vectorización e indexación.

In [ ]:
# Ejemplo de cómo podrías estructurar el script principal para compilar el proyecto
# Guardar este código como procesar_pdfs.py

"""
from procesar_pdfs import extract_text_from_pdfs, chunk_text, embedder, faiss, np, pickle

# Leer PDFs y procesar
pdf_files = [...]
pdf_texts = extract_text_from_pdfs(pdf_files)
all_chunks = []
for doc in pdf_texts:
    all_chunks.extend(chunk_text(doc))
embeddings = embedder.encode(all_chunks, show_progress_bar=True, convert_to_numpy=True)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
faiss.write_index(index, "faiss_index.bin")
with open("chunks.pkl", "wb") as f:
    pickle.dump(all_chunks, f)
"""

# 8. Ejemplo de Consulta al Modelo desde Otro Archivo

A continuación, se muestra cómo cargar el índice y los textos guardados para realizar preguntas y testear el sistema.

In [ ]:
# Cargar el índice y los chunks
index = faiss.read_index("c:/Users/TZ002NB/Desktop/rag-fine-tunning/faiss_index.bin")
with open("c:/Users/TZ002NB/Desktop/rag-fine-tunning/chunks.pkl", "rb") as f:
    all_chunks = pickle.load(f)

# Función para consultar el índice
def buscar_respuesta(pregunta, k=3):
    pregunta_vec = embedder.encode([pregunta], convert_to_numpy=True)
    D, I = index.search(pregunta_vec, k)
    resultados = [all_chunks[i] for i in I[0]]
    return resultados

# Ejemplo de consulta
pregunta = "¿Cuál es el tema principal de los documentos?"
respuestas = buscar_respuesta(pregunta)
for idx, resp in enumerate(respuestas):
    print(f"Respuesta {idx+1}:\n{resp}\n")